In [1]:
import numpy as np
import pandas as pd
from table2ascii import table2ascii, PresetStyle, Alignment
from IPython.core.display import HTML

In [2]:
players_df = pd.read_csv('https://raw.githubusercontent.com/NicolajBH/fpl-draft-update/main/draft_data.csv')
players_df.drop(columns=players_df.columns[0], inplace=True)

## Monthly standings df

In [3]:
# Monthly points
monthly_standings = players_df[['name', 'month', 'playerPoints']][(players_df['played'] == True)].groupby(['name', 'month']).sum('playerPoints').reset_index()
monthly_standings.reset_index(inplace=True)
monthly_standings.drop(columns=monthly_standings.columns[0], inplace=True)

# Cumulative sum overall points
total_points = players_df[players_df['played'] == True].groupby(['name', 'gw', 'month']).sum('playerPoints').reset_index()
total_points = total_points[['name', 'gw', 'month', 'playerPoints']]
total_points = total_points.rename(columns={'playerPoints':'GW Score'})
total_points['Total Points'] = total_points['GW Score'].groupby(total_points['name']).transform('cumsum')
total_points_eom = total_points.sort_values('Total Points', ascending=False).drop_duplicates(subset=['name','month'])

# Join the total points at the end of the month to the monthly standings df and add a monthly rank
monthly_standings = monthly_standings.merge(total_points_eom, left_on=['month', 'name'], right_on=['month', 'name'])
monthly_standings = monthly_standings.sort_values(['gw', 'playerPoints', 'Total Points'], ascending=[True, False, False])
monthly_standings['rank'] = monthly_standings.groupby('month')['playerPoints'].rank(method='first', ascending=False).astype(int)

# Format the dataframe
monthly_standings = monthly_standings.rename(columns={'playerPoints':'Month Score'}).reset_index()
monthly_standings.drop(columns=monthly_standings.columns[0], inplace=True)
monthly_standings = monthly_standings[['name', 'month', 'Month Score', 'rank', 'Total Points']]

# Add the top scoring player for each person every month
top_player = players_df.sort_values(['monthlyPoints','draft_rank'], ascending=[False, True])\
            .drop_duplicates(subset=('name', 'month'))
top_player = top_player.sort_values('deadline')
top_player = top_player[['web_name', 'short_name', 'month', 'name', 'monthlyPoints']]
monthly_standings = monthly_standings.merge(top_player, left_on=['name','month'], right_on=['name', 'month'])

## All months

In [4]:
monthly_standings

,name,month,Month Score,rank,Total Points,web_name,short_name,monthlyPoints
0,Ollie,August,275,1,275,Haaland,Haaland,58
1,Kris,August,233,2,233,Mitrović,Mitrović,34
2,Mattia,August,222,3,222,Kane,Kane,31
3,Jesus,August,217,4,217,Jesus,Jesus,32
4,Nicolaj,August,201,5,201,Salah,Salah,35
5,Nicolaj,September,93,1,294,Saka,Saka,16
6,Mattia,September,88,2,310,Kane,Kane,19
7,Jesus,September,78,3,295,Son,Son,19
8,Ollie,September,73,4,348,Trippier,Trippier,13
9,Kris,September,73,5,306,Toney,Toney,19


## Current Month

In [5]:
current_month = players_df.sort_values('deadline', ascending=False).values[0][12]
monthly_standings[monthly_standings.month == current_month]

,name,month,Month Score,rank,Total Points,web_name,short_name,monthlyPoints
35,Kris,March,164,1,1303,Jensen,Jensen,25
36,Mattia,March,151,2,1105,Salah,Salah,21
37,Ollie,March,128,3,1294,White,White,18
38,Jesus,March,124,4,1169,Saka,Saka,21
39,Nicolaj,March,115,5,1268,Mitoma,Mitoma,25


In [6]:
def player_top5_month(player):
    '''
    This function returns the top 5 players for the current month for player.
    player = ['Mattia', 'Ollie', 'Nicolaj', 'Kris', 'Jesus']
    '''
    player_top_5 = players_df[(players_df.month==current_month) & (players_df.name==player)]\
        .sort_values(['monthlyPoints','draft_rank'], ascending=[False, True])\
        .drop_duplicates(subset=['web_name']).head()
    player_top_5 = player_top_5[['web_name', 'monthlyPoints']].reset_index()
    player_top_5.drop(columns=player_top_5.columns[0], inplace=True)
    player_top_5.rename(columns={'web_name':'Player','monthlyPoints':'Points'}, inplace=True)
    player_top_5.index += 1
    player_top_5.style.set_caption(player)
    # Add MultiIndex Header
    player_top_5.columns = pd.MultiIndex.from_product([[player], player_top_5.columns])
    return player_top_5


def multi_table(table_list):
    ''' Acceps a list of IpyTable objects and returns a table which contains each IpyTable in a cell
    https://github.com/epmoyer/ipy_table/issues/24
    '''
    return HTML(
        '<table><tr style="background-color:white;">' + 
        ''.join(['<td>' + table._repr_html_() + '</td>' for table in table_list]) +
        '</tr></table>'
    )

multi_table([player_top5_month('Nicolaj'),\
            player_top5_month('Kris'),\
            player_top5_month('Ollie'),\
            player_top5_month('Jesus'),\
            player_top5_month('Mattia')])

## Most points left on the bench this season

In [7]:
bench_points = players_df[['name', 'playerPoints']][players_df['played'] == False].groupby('name').sum('playerPoints').sort_values('playerPoints', ascending=False)
bench_points = bench_points.rename(columns={'playerPoints':'Points On Bench'}).reset_index()
bench_points.index += 1
bench_points

,name,Points On Bench
1,Nicolaj,302
2,Kris,228
3,Jesus,227
4,Ollie,181
5,Mattia,149


## Number of times each person has finished in each position

In [8]:
names = list(monthly_standings.name.unique())
num_times_placed = pd.DataFrame()

for name in names:
    value_counts = monthly_standings[['rank']][monthly_standings['name'] == name].value_counts().rename_axis('rank').to_frame('counts')
    value_counts = value_counts.sort_values('rank').transpose()
    num_times_placed = pd.concat([num_times_placed, value_counts])
num_times_placed['name'] = names

num_times_placed = num_times_placed[['name',1,2,3,4,5]]
num_times_placed.rename(columns={1:'First', 2:'Second', 3:'Third', 4:'Fourth', 5:'Fifth'}, inplace=True)

num_times_placed.fillna(0, inplace=True)
num_times_placed.First, num_times_placed.Second, num_times_placed.Third, num_times_placed.Fourth, num_times_placed.Fifth = num_times_placed.First.astype(int), num_times_placed.Second.astype(int), num_times_placed.Third.astype(int), num_times_placed.Fourth.astype(int), num_times_placed.Fifth.astype(int)

num_times_placed.sort_values([(num_times_placed.columns[1]),(num_times_placed.columns[2]),(num_times_placed.columns[3]),(num_times_placed.columns[4]),(num_times_placed.columns[5])], ascending=False, inplace=True)
num_times_placed.reset_index(inplace=True)
num_times_placed.drop(columns=num_times_placed.columns[0], inplace=True)
num_times_placed.index += 1
num_times_placed

rank,name,First,Second,Third,Fourth,Fifth
1,Ollie,4,0,2,1,1
2,Kris,2,4,0,1,1
3,Nicolaj,2,1,2,1,2
4,Mattia,0,2,1,2,3
5,Jesus,0,1,3,3,1


## Best 3 and worst 3 gameweeks of the season

In [9]:
# TODO write better current_gw condition
current_gw = players_df.sort_values('gw', ascending=False).values[0][5]

gw_points = players_df[['name', 'gw', 'playerPoints']][players_df['played'] == True].groupby(['name', 'gw']).sum('playerPoints').reset_index()
gw_points = gw_points.rename(columns={"playerPoints":'Points'})
top3 = gw_points.sort_values('Points', ascending=False).head(3)
bottom3 = gw_points[(gw_points.Points != 0) & (gw_points.gw != current_gw)].sort_values('Points').head(3)

In [10]:
top3

,name,gw,Points
108,Nicolaj,25,101
130,Ollie,19,77
54,Kris,27,76


In [11]:
bottom3

,name,gw,Points
85,Nicolaj,2,19
66,Mattia,11,23
4,Jesus,5,23


## Best Transfers of the Season

In [12]:
# Best transfer defined as most points in the gameweek the player was transferred in
top5_transfers = players_df[(players_df['gw']*1.0) == players_df['bought']].sort_values('playerPoints', ascending=False).head(5)
top5_transfers = top5_transfers[['name', 'gameweek', 'web_name', 'playerPoints']]
top5_transfers

,name,gameweek,web_name,playerPoints
364,Nicolaj,25,Van Dijk,24
293,Nicolaj,20,Mahrez,19
1702,Ollie,2,Rodrigo,15
996,Kris,11,Mount,15
1950,Ollie,19,Leno,14


## Worst Transfers of the Season

In [13]:
worst5_transfers = players_df[(players_df['gw']*1.0) == players_df['bought']].sort_values('playerPoints').head(5)
worst5_transfers = worst5_transfers[['name', 'gameweek', 'web_name', 'playerPoints']]
worst5_transfers

,name,gameweek,web_name,playerPoints
527,Jesus,8,Collins,-3
127,Nicolaj,9,Sinisterra,-2
553,Jesus,9,Dalot,-2
389,Nicolaj,26,Shaw,-1
739,Jesus,22,Casemiro,-1


## Misc stats

In [14]:
def player_stats(name, stat):
    '''
    This function returns some stats for the person 'name'. The output is sorted based on the stat parameter.
    The stat can be appearances, ppg or playerPoints
    ppg = points per game
    appearances and playerPoints don't consider whether player was in starting 11 or not, only if they were in the 15 man squad
    '''
    appearances_df = players_df[['web_name']][players_df['name']==name].value_counts().rename_axis('web_name').to_frame('counts')
    points_df = players_df[['web_name', 'playerPoints']][players_df['name']==name].groupby('web_name').sum().sort_values(by='playerPoints', ascending=False)
    ppg_df = appearances_df.merge(points_df, left_on='web_name', right_on='web_name')
    ppg_df['ppg'] = ppg_df['playerPoints'] / ppg_df['counts']
    ppg_df.rename(columns={ppg_df.columns[0]:"appearances"}, inplace=True)
    ppg_df.sort_values(stat, ascending=False, inplace=True)
    ppg_df.reset_index(inplace=True)
    ppg_df.index += 1
    ppg_df.columns = pd.MultiIndex.from_product([[name], ppg_df.columns])
    return ppg_df

In [15]:
# stat parameter can be 'ppg', 'appearances', 'playerPoints'
multi_table([player_stats('Nicolaj', 'appearances').head(),\
             player_stats('Kris', 'appearances').head(),\
             player_stats('Ollie', 'appearances').head(),\
             player_stats('Jesus', 'appearances').head(),\
             player_stats('Mattia', 'appearances').head()])

## Find number of transfers for each player

In [16]:
playerIdList = list(players_df.playerId.unique())
bought = []
sold = []
for playerId in playerIdList:
    bought.append(len(players_df[players_df['playerId']==playerId].bought.unique()))
    sold.append(len(players_df[players_df['playerId']==playerId].sold.dropna().unique())) # drop nan values so they dont count as sold

num_transfers_df = pd.DataFrame({
    'playerId':playerIdList,
    'bought':bought,
    'sold':sold,
})
num_transfers_df = pd.merge(num_transfers_df,players_df[['playerId','web_name']],on='playerId',how='left')
num_transfers_df.drop_duplicates(inplace=True)

num_transfers_df.sort_values('bought', ascending=False).head(5)

,playerId,bought,sold,web_name
811,210,5,4,Mitrović
1289,116,4,3,Mac Allister
441,594,4,3,Isak
837,332,4,3,Shaw
125,303,4,4,Mahrez
